First is we need to import the libraries and tools needed.

In [19]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import cv2
import os

from keras.backend import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import Sequential

After that, we need to import the dataset/s that will be used in this exercise. My files were saved in my google drive so I mounted it on this program.

In [24]:
from google.colab import drive
drive.mount('drive') 

train_data = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

train_path = '/content/drive/MyDrive/COE 005/PE/train'
train_dataset = train_data.flow_from_directory(train_path, 
                                          target_size = (224, 224), 
                                          batch_size = 32, 
                                          class_mode = 'categorical')

Mounted at drive
Found 249 images belonging to 3 classes.


In [25]:
test_data = ImageDataGenerator(rescale=1./255)
test_path = '/content/drive/MyDrive/COE 005/PE/test'
test_dataset = test_data.flow_from_directory(test_path, 
                                         target_size = (224, 224), 
                                         batch_size = 32, 
                                         class_mode = 'categorical')

Found 65 images belonging to 3 classes.


We'll be using the sequential model in keras.

In [26]:
model = Sequential()
model.add(Conv2D(32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=(224,224,3)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

next is fitting the data in 5 epochs that have 10 steps each to complete

In [30]:
history = model.fit_generator(train_dataset,steps_per_epoch=7,epochs = 10,validation_data = test_dataset,validation_steps=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
7/7 [==============================] - 49s 7s/step - loss: 0.4247 - accuracy: 0.8525 - val_loss: 0.5527 - val_accuracy: 0.7656
Epoch 2/10
7/7 [==============================] - 47s 7s/step - loss: 0.5428 - accuracy: 0.7926 - val_loss: 0.6218 - val_accuracy: 0.7031
Epoch 3/10
7/7 [==============================] - 50s 7s/step - loss: 0.4330 - accuracy: 0.8157 - val_loss: 0.5301 - val_accuracy: 0.7969
Epoch 4/10
7/7 [==============================] - 47s 7s/step - loss: 0.4121 - accuracy: 0.8295 - val_loss: 0.5605 - val_accuracy: 0.7344
Epoch 5/10
7/7 [==============================] - 49s 7s/step - loss: 0.4452 - accuracy: 0.7788 - val_loss: 0.5159 - val_accuracy: 0.8281
Epoch 6/10
7/7 [==============================] - 47s 7s/step - loss: 0.4941 - accuracy: 0.7834 - val_loss: 0.5606 - val_accuracy: 0.7812
Epoch 7/10
7/7 [==============================] - 47s 7s/step - loss: 0.4243 - accuracy: 0.8341 - val_loss: 0.4720 - val_accuracy: 0.8750
Epoch 8/10
7/7 [==================

In [31]:
#getting the accuracy
accuracy = model.evaluate(train_dataset)
print('Test accuracy:', accuracy[1])

8/8 [==============================] - 19s 2s/step - loss: 0.3206 - accuracy: 0.9076
Test accuracy: 0.9076305031776428


The accuracy I got is 90.7%. Now we'll be using the model for validation.

In [61]:
dataset = '/content/drive/MyDrive/COE 005/PE/'

def loadImages(path, folder):
    data_files = sorted([os.path.join(path, folder, file)
                        for file in os.listdir(path + folder + '')
                        if (file.endswith('.jpg') or file.endswith('.jpeg'))])
    return data_files

validation = loadImages(dataset, 'validation')

g = validation
f = []
images = []

for x in g:
    img = cv2.imread(x)
    imeg = img[:, :, :3]
    images.append(imeg)

    width = 224
    height = 224
    dim = (width, height)

    resized = cv2.resize(imeg, dim, interpolation=cv2.INTER_AREA)
    f.append(resized)

f = np.array(f)
f = f/255

predict = model.predict(f)
predictions = np.argmax(predict, axis=1)
print(predictions)

1/1 [==============================] - 0s 445ms/step
[1 1 0 2 2 2 0 0 2 0]


In [62]:
from google.colab import drive
drive.mount('drive') 

predict.tofile('/content/drive/MyDrive/COE 005/PE/danao_cnn_output_ex.csv')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
